<a href="https://colab.research.google.com/github/CrocusBehemoth/ML_course_Pavia_23/blob/newCrocus/EntDetect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import random
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")
torch.set_default_device(device)

Using cuda device


We want to implement a Neural Network to detect entanglement in a two-qubits state density matrix.

In [2]:
def rand_angles():
  theta=np.random.rand()*2*np.pi
  phi=np.random.rand()*np.pi*np.sin(theta)
  return theta,phi

def generate_state(theta,phi,vec1,vec2):
  psi=np.cos(theta/2.)*vec1 + np.sin(theta/2.)*np.exp(1j*phi)*vec2
  return np.outer(psi,psi.conj())

def generate_rand_dens(n):
  #genero matrice hermitiana
  random_hermitian = np.random.randn(2*n, 2*n) + 1j * np.random.randn(2*n,2*n)
  random_hermitian = (random_hermitian + random_hermitian.conj().T) / 2  # Facciamola hermitiana!

  # prendo valore assoluto autovalori
  eigenvalues, eigenvectors = np.linalg.eigh(random_hermitian)
  positive_eigenvalues = np.abs(eigenvalues)
  positive_hermitian = eigenvectors @ np.diag(positive_eigenvalues) @ eigenvectors.conj().T

  # Normalizzo a traccia 1
  density_matrix = positive_hermitian / np.trace(positive_hermitian)
  return density_matrix

def Partial_transpose(Mat):
  M=np.copy(Mat)
  temp=M[0,1]
  M[0,1]=M[1,0]
  M[1,0]=temp
  temp=M[3,0]
  M[3,0]=M[2,1]
  M[2,1]=temp
  temp=M[2,3]
  M[2,3]=M[3,2]
  M[3,2]=temp
  temp=M[0,3]
  M[0,3]=M[1,2]
  M[1,2]=temp
  return M

def Ent_check(M):
  M=Partial_transpose(M)
  eigenvalues, _ = np.linalg.eigh(M)
  if min(eigenvalues) < 0:
    return 1
  else:
    return 0

def Add_noise(M):
  p=np.random.rand(1)
  I=np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]])
  return p*M+(1-p)*I/4

def Gen_pure_state():
  v=np.random.rand(4,1)+1j*np.random.rand(4,1)
  v=v/np.linalg.norm(v)
  #v=v.reshape((-1,1))
  v=np.outer(v,v.conj())
  return v

def von_neumann_entropy(M):
    eigenvalues, _ = np.linalg.eigh(M)  # Eigenvalues of the density matrix
    non_zero_eigenvalues = eigenvalues[eigenvalues > 1e-10]  # Remove small values
    entropy_terms = -non_zero_eigenvalues * np.log2(non_zero_eigenvalues)
    entropy = np.sum(entropy_terms)
    return entropy

def Eliminate_Off_diag(M,a1,a2,b1,b2):
  v1=np.array([a1[0]*b1[0],a1[0]*b1[1],a1[1]*b1[0],a1[1]*b1[1]])
  v2=np.array([a1[0]*b2[0],a1[0]*b2[1],a1[1]*b2[0],a1[1]*b2[1]])
  v3=np.array([a2[0]*b1[0],a2[0]*b1[1],a2[1]*b1[0],a2[1]*b1[1]])
  v4=np.array([a2[0]*b2[0],a2[0]*b2[1],a2[1]*b2[0],a2[1]*b2[1]])
  m11=np.dot(v1.conj(),np.dot(M,v1))
  m22=np.dot(v2.conj(),np.dot(M,v2))
  m33=np.dot(v3.conj(),np.dot(M,v3))
  m44=np.dot(v4.conj(),np.dot(M,v4))
  return m11*np.outer(v1,v1.conj())+m22*np.outer(v2,v2.conj())+m33*np.outer(v3,v3.conj())+m44*np.outer(v4,v4.conj())

def Mutual_correlation(M,a1,a2,b1,b2):
  s=von_neumann_entropy(M)
  M=Eliminate_Off_diag(M,a1,a2,b1,b2)
  mc=von_neumann_entropy(M)-s
  return mc
def calculate_features(M):
  d=np.array([1,0])
  u=np.array([0,1])
  rplus=(d+u)/np.sqrt(2)
  rminus=(d-u)/np.sqrt(2)
  splus=(d+1j*u)/np.sqrt(2)
  sminus=(d-1j*u)/np.sqrt(2)
  m1=Mutual_correlation(M,d,u,rplus,rminus)
  m2=Mutual_correlation(M,splus,sminus,rplus,rminus)
  m3=Mutual_correlation(M,splus,sminus,d,u)
  m4=Mutual_correlation(M,d,u,d,u)
  return m1, m2, m3, m4

In [3]:
# Specify the CSV file name
csv_file_name = 'matrices.csv'


In [4]:


n_densitymat=50000

dd=np.array([1,0,0,0])
du=np.array([0,1,0,0])
ud=np.array([0,0,1,0])
uu=np.array([0,0,0,1])

tot_mat=np.zeros(1)
label=np.zeros([7*n_densitymat,1])
features=np.zeros([7*n_densitymat,4])
for i in range(0,n_densitymat):
  s=0
  theta,phi=rand_angles()
  mat=generate_state(theta,phi,du,ud)
  mat=Add_noise(mat)
  label[i*7+s]=Ent_check(mat)
  features[i*7+s,:]=calculate_features(mat)
  s+=1
  theta,phi=rand_angles()
  mat=generate_state(theta,phi,dd,uu)
  mat=Add_noise(mat)
  label[i*7+s]=Ent_check(mat)
  features[i*7+s,:]=calculate_features(mat)
  s+=1
  mat=generate_rand_dens(2)
  mat=Add_noise(mat)
  label[i*7+s]=Ent_check(mat)
  features[i*7+s,:]=calculate_features(mat)
  s+=1
  mat=np.kron(generate_rand_dens(1),generate_rand_dens(1))
  mat=Add_noise(mat)
  label[i*7+s]=Ent_check(mat)
  features[i*7+s,:]=calculate_features(mat)
  s+=1
  mat=Gen_pure_state()
  mat=Add_noise(mat)
  label[i*7+s]=Ent_check(mat)
  features[i*7+s,:]=calculate_features(mat)
  s+=1
  mat=generate_state(np.pi/2,0,dd,uu)
  mat=Add_noise(mat)
  label[i*7+s]=Ent_check(mat)
  features[i*7+s,:]=calculate_features(mat)
  s+=1
  mat=generate_state(np.pi/2,0,du,ud)
  mat=Add_noise(mat)
  label[i*7+s]=Ent_check(mat)
  features[i*7+s,:]=calculate_features(mat)
  print(str((i/n_densitymat)*100)+"%")

print("shape features="+str(np.shape(features)))
print("shape labels="+str(np.shape(label)))
df=pd.DataFrame(np.hstack((features,label)),columns=["C1","C2","C3","C4","Labels"])
df.to_csv(csv_file_name,mode='w')

Output streaming troncato alle ultime 5000 righe.
90.00399999999999%
90.006%
90.008%
90.01%
90.012%
90.01400000000001%
90.01599999999999%
90.018%
90.02%
90.022%
90.024%
90.026%
90.02799999999999%
90.03%
90.032%
90.034%
90.036%
90.038%
90.03999999999999%
90.042%
90.044%
90.046%
90.048%
90.05%
90.05199999999999%
90.054%
90.056%
90.058%
90.06%
90.062%
90.064%
90.066%
90.068%
90.07%
90.072%
90.074%
90.076%
90.078%
90.08%
90.082%
90.084%
90.086%
90.08800000000001%
90.09%
90.092%
90.094%
90.096%
90.098%
90.10000000000001%
90.102%
90.104%
90.106%
90.108%
90.11%
90.11200000000001%
90.114%
90.116%
90.118%
90.12%
90.122%
90.12400000000001%
90.12599999999999%
90.128%
90.13%
90.132%
90.134%
90.13600000000001%
90.13799999999999%
90.14%
90.142%
90.144%
90.146%
90.148%
90.14999999999999%
90.152%
90.154%
90.156%
90.158%
90.16%
90.16199999999999%
90.164%
90.166%
90.168%
90.16999999999999%
90.172%
90.17399999999999%
90.176%
90.178%
90.18%
90.182%
90.184%
90.18599999999999%
90.188%
90.19%
90.192000000000

In [ ]:
from torch.utils.data import Dataset, DataLoader

class Dati(Dataset):
    def __init__(self, csv_file,device):
        self.data = pd.read_csv(csv_file)
        self.features = torch.from_numpy(self.data.iloc[:, 1:5].values).to(device)
        self.labels = torch.from_numpy(self.data.iloc[:, 5].values).to(device)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        feature = torch.tensor(self.features[idx], dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        return feature, label

In [ ]:
class DNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(DNN, self).__init__()
        #definisci sequenza operazioni NN
        self.model = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.Linear(hidden_size, output_size),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x).squeeze()

class DNNReLU(nn.Module):
    def __init__(self, input_size, hidden_size1,hidden_size2, output_size):
        super(DNNReLU, self).__init__()
        #definisci sequenza operazioni NN
        self.model = nn.Sequential(
            nn.Linear(input_size, hidden_size1),
            nn.ReLU(),
            nn.Linear(hidden_size1,hidden_size2),
            nn.ReLU(),
            nn.Linear(hidden_size2, output_size),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x).squeeze()

In [ ]:
model=DNN(2,2,2)
print("Ecco il nostro bello modello:{model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Ecco il nostro bello modello:{model}


Layer: model.0.weight | Size: torch.Size([2, 2]) | Values : tensor([[ 0.0971,  0.5499],
        [-0.5122,  0.5230]], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: model.0.bias | Size: torch.Size([2]) | Values : tensor([-0.6900, -0.0556], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: model.1.weight | Size: torch.Size([2, 2]) | Values : tensor([[ 0.2441, -0.3415],
        [ 0.2722,  0.3520]], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: model.1.bias | Size: torch.Size([2]) | Values : tensor([0.2900, 0.1777], device='cuda:0', grad_fn=<SliceBackward0>) 



In [ ]:
x = torch.ones(4)  # input tensor
y = torch.zeros(1)  # expected output
w = torch.randn(4, 1, requires_grad=True)
b = torch.randn(1, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

In [ ]:
batch_size=64
split=0.7
epoch=100
learning_rate=1e-4
data=Dati(csv_file_name,device)

training_data, test_data = torch.utils.data.random_split(data, [int(split*len(data)), len(data)-int(split*len(data))],generator=torch.Generator(device))
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)


In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += torch.sum(torch.where((torch.round(pred)==y), 1, 0).float())

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return float(100*correct)


In [ ]:
# Initialize the loss function
model=DNN(4,50,1).to(device)
modelReLU=DNNReLU(4,25,50,1).to(device)
loss_fn = nn.BCELoss().to(device)
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
accuracy=torch.Tensor(2,epoch)
print(accuracy.shape)
for t in range(epoch):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    accuracy[0,t]=test_loop(test_dataloader, model, loss_fn)
print("Done for linear!")
for t in range(epoch):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, modelReLU, loss_fn, optimizer)
    accuracy[1,t]=test_loop(test_dataloader, modelReLU, loss_fn)
print("Done for ReLu!")


NameError: ignored

In [ ]:
ntests=1000
v=torch.tensor([ntests,3])
t=torch.tensor([ntests,2])
for i in range(0,ntests-1):
  M=generate_rand_dens(2)
  v[i,0]=EntCheck(M)
  features=calculate_features(M)
  v[i,1]=torch.float((features[1]+[features[2]+features[3]-features[4]<4))
  v[i,1]=torch.round(model(features))
  v[i,2]=torch.round(modelReLU(features))

t=[v[:,1]-v[:,0],v[:,2]-v[:,0],v[:3]-v[:,0]]